## Running the fluorescent survival pipeline

Run the first cell (click it and press shift+enter.) Select your experiment directory from the file browser.

Run the second cell. When complete, check your experiment's results directory.

In [ ]:
%run utils/experiment_chooser.ipynb
experiment_chooser.choose()

In [ ]:
from improc.processes import Filter, Pipeline, BaSiC, Stitch, Stack, Measurement, Measurements
from improc.processes.flatfield import RollingBall
from improc.processes.legacy_survival.analysis import SurvivalAnalysis
from improc.experiment.types import Channel, Timepoint, Exposure

INPUT_DATASET = "raw_imgs"

try:
    from config.local import ILASTIK_BINARY, ILASTIK_CLASSIFIERS, MODEL_DIR, RSCRIPT_DIR
except Exception:
    from config.default import ILASTIK_BINARY, ILASTIK_CLASSIFIERS, MODEL_DIR, RSCRIPT_DIR

model_path = MODEL_DIR / "GFP_rat_neuron_disc.h5"
survival_script = RSCRIPT_DIR / "survival.R"

measurements_to_take = [
    Measurement(
        channel=Channel.GFP,
        mean=True,
        std=True,
        median=True,
        pct_95=True,
        area=True,
        perimeter=True,
        centroid=True,
        skew=True,
        kurtosis=True,
        gradient_mean=True,
        gradient_std=True,
        aspect_ratio=True
    )
]

pipeline = Pipeline(
    RollingBall(radius=12),
    Stitch(),
    Stack(),
    SurvivalAnalysis(model_path, survival_script),
    Measurements(measurements_to_take)
)

experiment = experiment_chooser.fetch()